In [1]:
import os
%pwd

'/Applications/AI/GAN/research'

In [2]:
os.chdir("../")
%pwd

'/Applications/AI/GAN'

In [ ]:
# GAN Model Evaluation: entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class GANEvaluationConfig:
    generator_model_path: Path
    discriminator_model_path: Path
    test_data_path: Path
    evaluation_output_path: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int
    num_samples: int
    save_format: str

# GAN Model Evaluation: config manager
from src.GAN.constants import *
from src.GAN.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_gan_evaluation_config(self) -> GANEvaluationConfig:
        eval_config = GANEvaluationConfig(
            generator_model_path=Path(self.config.gan_training.trained_generator_path),
            discriminator_model_path=Path(self.config.gan_training.trained_discriminator_path),
            test_data_path=Path(self.config.data_ingestion.unzip_dir),
            evaluation_output_path=Path(self.config.evaluation.output_path),
            mlflow_uri=self.config.evaluation.mlflow_uri,
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            num_samples=self.params.EVAL_SAMPLES,
            save_format=self.params.SAVE_FORMAT
        )
        return eval_config

# GAN Model Evaluation: component
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from scipy.linalg import sqrtm
from tqdm import tqdm
from src.GAN import logger

class GANEvaluation:
    def __init__(self, config: GANEvaluationConfig):
        self.config = config
        create_directories([self.config.evaluation_output_path])
        
    def load_models(self):
        """Load generator and discriminator models"""
        logger.info("Loading GAN models")
        self.generator = tf.keras.models.load_model(self.config.generator_model_path)
        self.discriminator = tf.keras.models.load_model(self.config.discriminator_model_path)
    
    def load_test_data(self):
        """Load test images for evaluation"""
        logger.info("Loading test data")
        
        # Find paths containing photos and Monet images
        photo_dir = self.find_directory_with_images(self.config.test_data_path, "photo")
        monet_dir = self.find_directory_with_images(self.config.test_data_path, "monet")
        
        if not photo_dir or not monet_dir:
            raise ValueError("Could not find photo and Monet directories")
        
        # Load a batch of test photos
        photo_files = self.get_image_files(photo_dir)
        self.test_photos = self.load_images(
            photo_files[:self.config.num_samples],
            self.config.params_image_size[:2]
        )
        
        # Load real Monet paintings for comparison
        monet_files = self.get_image_files(monet_dir)
        self.real_monet = self.load_images(
            monet_files[:self.config.num_samples],
            self.config.params_image_size[:2]
        )
    
    def find_directory_with_images(self, root_dir, name_contains):
        """Find a directory that contains images and has 'name_contains' in its path"""
        for root, dirs, files in os.walk(root_dir):
            image_files = [f for f in files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            if image_files and name_contains.lower() in root.lower():
                return root
        return None
    
    def get_image_files(self, directory):
        """Get all image files in a directory"""
        extensions = ['.jpg', '.jpeg', '.png']
        files = []
        for ext in extensions:
            files.extend([os.path.join(directory, f) for f in os.listdir(directory)
                         if f.lower().endswith(ext)])
        return files
    
    def load_images(self, file_paths, image_size):
        """Load and preprocess images from file paths"""
        images = []
        for path in file_paths:
            img = tf.io.read_file(path)
            img = tf.io.decode_jpeg(img, channels=3)
            img = tf.image.resize(img, image_size)
            img = (tf.cast(img, tf.float32) / 127.5) - 1  # Normalize to [-1, 1]
            images.append(img)
        
        if images:
            return tf.stack(images)
        return None
    
    def generate_samples(self):
        """Generate Monet-style images from test photos"""
        logger.info("Generating Monet-style samples")
        self.generated_images = self.generator(self.test_photos, training=False)
        return self.generated_images
    
    def calculate_fid(self, real_images, generated_images):
        """Calculate Fréchet Inception Distance between real and generated images"""
        logger.info("Calculating FID score")
        
        # Load InceptionV3 model
        inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))
        
        # Resize and preprocess images for InceptionV3
        def preprocess_for_inception(images):
            images = tf.image.resize(images, (299, 299))
            images = (images + 1) * 127.5  # Convert from [-1, 1] to [0, 255]
            images = preprocess_input(images)  # Inception preprocessing
            return images
        
        real_preprocessed = preprocess_for_inception(real_images)
        generated_preprocessed = preprocess_for_inception(generated_images)
        
        # Extract features
        real_features = inception_model.predict(real_preprocessed)
        generated_features = inception_model.predict(generated_preprocessed)
        
        # Calculate mean and covariance
        mu1, sigma1 = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
        mu2, sigma2 = generated_features.mean(axis=0), np.cov(generated_features, rowvar=False)
        
        # Calculate square root of product of covariances
        ssdiff = np.sum((mu1 - mu2) ** 2.0)
        covmean = sqrtm(sigma1.dot(sigma2))
        
        # Check and correct imaginary numbers from sqrt
        if np.iscomplexobj(covmean):
            covmean = covmean.real
        
        # Calculate FID
        fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
        return fid
    
    def save_generated_images(self):
        """Save generated images for visual inspection"""
        logger.info("Saving generated images")
        
        save_dir = os.path.join(self.config.evaluation_output_path, "generated_samples")
        create_directories([save_dir])
        
        # Create a grid of original vs generated images
        plt.figure(figsize=(20, 10))
        
        for i in range(min(10, len(self.test_photos))):
            # Display original photo
            plt.subplot(2, 10, i + 1)
            plt.title("Original")
            photo = (self.test_photos[i] + 1) * 0.5  # Denormalize
            plt.imshow(photo)
            plt.axis('off')
            
            # Display generated Monet-style image
            plt.subplot(2, 10, i + 11)
            plt.title("Generated")
            generated = (self.generated_images[i] + 1) * 0.5  # Denormalize
            plt.imshow(generated)
            plt.axis('off')
        
        comparison_path = os.path.join(save_dir, "comparison.png")
        plt.savefig(comparison_path)
        plt.close()
        
        # Save individual generated images
        for i, img in enumerate(self.generated_images):
            img = (img + 1) * 127.5  # Convert from [-1, 1] to [0, 255]
            img = tf.cast(img, tf.uint8)
            img_path = os.path.join(save_dir, f"generated_{i}.{self.config.save_format}")
            
            if self.config.save_format == 'png':
                tf.io.write_file(img_path, tf.io.encode_png(img))
            else:
                tf.io.write_file(img_path, tf.io.encode_jpeg(img))
        
        return comparison_path, save_dir
    
    # Add this method to your GANEvaluation class
    def save_metrics_json(self, metrics_dict, filepath):
        """Save metrics to JSON file without using the problematic save_json function"""
        import json
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath, 'w') as f:
            json.dump(metrics_dict, f, indent=4)
        logger.info(f"Metrics saved to {filepath}")
    
    def evaluate(self):
        """Evaluate GAN performance"""
        self.load_models()
        self.load_test_data()
        self.generate_samples()
        
        # Calculate FID score
        try:
            self.fid_score = self.calculate_fid(self.real_monet, self.generated_images)
            logger.info(f"FID Score: {self.fid_score}")
        except Exception as e:
            logger.error(f"Error calculating FID score: {e}")
            self.fid_score = None
        
        # Save generated images
        self.comparison_path, self.samples_dir = self.save_generated_images()
        
        # Save scores directly using json module
        import json
        scores = {"fid_score": float(self.fid_score) if self.fid_score is not None else None}
        metrics_path = os.path.join(self.config.evaluation_output_path, "metrics.json")
        os.makedirs(os.path.dirname(metrics_path), exist_ok=True)
        with open(metrics_path, 'w') as f:
            json.dump(scores, f, indent=4)
        logger.info(f"Metrics saved to {metrics_path}")
        
        return scores
    
    def log_into_mlflow(self):
        """Log evaluation metrics and artifacts to MLflow with better error handling"""
        logger.info("Attempting to log to MLflow")
        
        # Skip MLflow logging if no URI is configured
        if not self.config.mlflow_uri:
            logger.info("No MLflow URI configured, skipping MLflow logging")
            return
        
        try:
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            
            # Only proceed with MLflow logging if proper URI is configured
            if tracking_url_type_store not in ["http", "https"]:
                logger.warning(f"MLflow tracking URI scheme '{tracking_url_type_store}' not supported for artifact logging")
                logger.info("To use MLflow, configure a valid tracking server with http/https")
                return
            
            with mlflow.start_run():
                # Log parameters
                mlflow.log_params(self.config.all_params)
                
                # Log metrics
                if self.fid_score is not None:
                    mlflow.log_metric("fid_score", self.fid_score)
                
                # Log artifacts and models
                mlflow.log_artifact(self.comparison_path)
                mlflow.keras.log_model(self.generator, "generator_model")
                mlflow.keras.log_model(self.discriminator, "discriminator_model")
                
                logger.info("Successfully logged to MLflow")
                
        except Exception as e:
            logger.warning(f"Error logging to MLflow: {e}")
            logger.info("Evaluation results are still saved locally")

# GAN Model Evaluation: pipeline
try:
    config = ConfigurationManager()
    eval_config = config.get_gan_evaluation_config()
    evaluation = GANEvaluation(eval_config)
    
    scores = evaluation.evaluate()
    
    # Try MLflow logging but continue if it fails
    try:
        evaluation.log_into_mlflow()
    except Exception as e:
        logger.warning(f"MLflow logging failed: {e}")
    
    logger.info(f"Evaluation completed successfully! Metrics: {scores}")
except Exception as e:
    logger.exception(f"Error during GAN evaluation: {e}")
    raise e

/Applications/AI/GAN/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[2025-03-20 11:28:50,714: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-20 11:28:50,717: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-20 11:28:50,718: INFO: common: created directory at: artifacts]
[2025-03-20 11:28:50,719: INFO: common: created directory at: artifacts/evaluation]
[2025-03-20 11:28:50,719: INFO: 4243899433: Loading GAN models]
[2025-03-20 11:28:51,229: INFO: 4243899433: Loading test data]
[2025-03-20 11:28:51,397: INFO: 4243899433: Generating Monet-style samples]
[2025-03-20 11:28:54,356: INFO: 4243899433: Calculating FID score]
1/1 [==============================] - 3s 3s/step
[2025-03-20 11:29:23,987: INFO: 4243899433: FID Score: 458.2857848227136]
[2025-03-20 11:29:23,992: INFO: 4243899433: Saving generated images]
[2025-03-20 11:29:24,002: INFO: common: created directory at: artifacts/evaluation/generated_samples]
[2025-03-20 11:29:24,811: INFO: 4243899433: Metrics saved to artifacts/evaluation/metrics.json]
[2025-0